In [7]:
import numpy as np

class MatrixFactorization:
    def __init__(self):
        self.M = None  # The input matrix to be factorized
        self.k = None  # The target lower dimension
        self.A = None  # The left factor matrix
        self.B = None  # The right factor matrix
        self.frontier = []  # The frontier for search

    def read_matrix(self):
        """
        Read the input matrix from the user.
        """
        n = int(input("Enter the number of rows and columns of the matrix: "))
        matrix = []
        print("Enter the matrix elements row by row, separated by spaces:")
        for _ in range(n):
            row = [float(x) for x in input().split()]
            matrix.append(row)
        self.M = np.array(matrix)

    def initialize_factors(self):
        """
        Initialize the factor matrices A and B with random values,
        and add the initial node to the frontier.
        """
        m, n = self.M.shape
        self.A = np.random.rand(m, self.k)
        self.B = np.random.rand(self.k, n)
        self.frontier.append(Node(self.A, self.B, self.calculate_cost(self.M, self.A, self.B)))

    def factorize_matrix(self):
        """
        Perform matrix factorization by searching for the optimal A and B matrices.
        """
        while self.frontier:
            self.frontier.sort(key=lambda node: node.cost, reverse=True)
            current_node = self.frontier.pop()
            if len(current_node.A[0]) == self.k:
                self.A, self.B = current_node.A, current_node.B
                return
            self.frontier.extend(self.expand_node(current_node, self.M))

    def expand_node(self, node, M):
        """
        Expand the current node by adding new rows to A or B matrices,
        and generate new child nodes.
        """
        children = []
        m, n = M.shape
        k = len(node.A[0])
        for i in range(k + 1, m + 1):
            A_new = np.vstack((node.A, np.random.rand(1, k)))
            B_new = np.linalg.lstsq(A_new, M)[0]
            cost = self.calculate_cost(M, A_new, B_new)
            children.append(Node(A_new, B_new, cost, node))
        for i in range(k + 1, n + 1):
            B_new = np.vstack((node.B, np.random.rand(k, 1)))
            A_new = np.linalg.lstsq(B_new.T, M.T)[0].T
            cost = self.calculate_cost(M, A_new, B_new)
            children.append(Node(A_new, B_new, cost, node))
        return children

    @staticmethod
    def calculate_cost(M, A, B):
        """
        Calculate the cost (Frobenius norm) of the difference between
        the input matrix M and the product of A and B.
        """
        return np.linalg.norm(M - np.dot(A, B))

class Node:
    def __init__(self, A, B, cost, parent=None):
        self.A = A  # Left factor matrix
        self.B = B  # Right factor matrix
        self.cost = cost  # Cost of the current node
        self.parent = parent  # Parent node in the search tree



In [8]:
mf = MatrixFactorization()
mf.read_matrix()
mf.k = int(input("Enter the target lower dimension (k): "))
mf.initialize_factors()
mf.factorize_matrix()
print("Matrix M:")
print(mf.M)
print("\nMatrix A:")
print(mf.A)
print("\nMatrix B:")
print(mf.B)

Enter the number of rows and columns of the matrix: 3
Enter the matrix elements row by row, separated by spaces:
1 2 3
4 5 6
7 8 9
Enter the target lower dimension (k): 2
Matrix M:
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]

Matrix A:
[[0.45104541 0.38902192]
 [0.11200781 0.79480547]
 [0.99737458 0.92192382]]

Matrix B:
[[0.32981041 0.70096748 0.73555867]
 [0.76552979 0.36952864 0.19630254]]
